# Specifications of quantification specific subclasses

> This module defines the specifications of quantification related subclasses of `ProcessingStrategy` and `ProcessingObject`

In [ ]:
#| default_exp quantification/specs

In [ ]:
#| export

from abc import abstractmethod
from typing import Dict, List
from skimage import io

from findmycells.core import ProcessingObject, ProcessingStrategy
from findmycells.database import Database
from findmycells import utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class QuantificationStrategy(ProcessingStrategy):
    
    @property
    def processing_type(self):
        return 'quantification'


    @abstractmethod
    def _add_quantification_results_to_database(quantification_object: ProcessingObject, results: Dict) -> ProcessingObject:
        # This method can actually just be copy-pasted into each strategy that inherits from QuantificationStrategy
        # However, it is implemented here as abstractmethod to ensure that it will be raise an error if it´s overlooked.
        # For implementation, the code in the docstring below can simply be copy pasted into each subclass,
        # and the following line needs to be added as final line to the run() method:
        # quantification_object = self._add_quantification_results_to_database(quantification_object = quantification_object, results = quantification_results)
        """
        if hasattr(quantification_object.database, 'quantification_results') == False:
            setattr(quantification_object.database, 'quantification_results', {})
        if self.__class__.__name__ not in quantification_object.database.quantification_results.keys():
            quantification_object.database.quantification_results[self.__class__.__name__] = {}
        quantification_object.database.quantification_results[self.__class__.__name__][quantification_object.file_id] = results
        """
        return quantification_object

In [ ]:
#| export

class QuantificationObject(ProcessingObject):
    
    def __init__(self, database: Database, file_ids: List[str], strategies: List[QuantificationStrategy]) -> None:
        super().__init__(database = database, file_ids = file_ids, strategies = strategies)
        self.file_id = file_ids[0]
        self.segmentations_per_area_roi_id = self._load_postprocessed_segmentations()


    @property
    def processing_type(self):
        return 'quantification'


    def _load_postprocessed_segmentations(self) -> Dict:
        segmentations_per_area_roi_id = dict()
        for elem in self.database.quantified_segmentations_dir.iterdir():
            if elem.is_dir():
                filenames = [filename for filename in listdir_nohidden(elem) if filename.startswith(self.file_id)]
                if len(filenames) > 0:
                    area_roi_id = elem.name
                    segmentations_per_area_roi_id[area_roi_id] = utils.load_zstack_as_array_from_single_planes(path = elem, file_id = self.file_id)
        return segmentations_per_area_roi_id
            

    def save_postprocessed_segmentations(self) -> None:
        for area_roi_id in self.segmentations_per_area_roi_id.keys():
            for plane_index in range(self.segmentations_per_area_roi_id[area_roi_id].shape[0]):
                image = self.segmentations_per_area_roi_id[area_roi_id][plane_index]
                filepath = self.database.quantified_segmentations_dir.joinpath(area_roi_id)
                if filepath.is_dir() == False:
                    filepath.mkdir()
                filename_path = filepath.joinpath(f'{self.file_id}-{str(plane_index).zfill(3)}_postprocessed_segmentations.png')
                io.imsave(filename_path, image, check_contrast=False)


    def add_processing_specific_infos_to_updates(self, updates: Dict) -> Dict:
        return updates

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()